# Implementación Path Relinking
---

In [17]:
from structure import instance, solution
from algorithms import grasp
from structure.solution import evaluate
from copy import deepcopy
import pandas as pd
import random

In [18]:
path = "instances/MDG-a_12_100_m10.txt"
inst = instance.readInstance(path)
sol, conjunto_soluciones, tiempo_final_grasp = grasp.execute(inst, 1, 0.1)

In [19]:
conjunto_soluciones_pd = pd.DataFrame(conjunto_soluciones)
conjunto_soluciones_pd = conjunto_soluciones_pd.drop_duplicates("sol")

In [20]:
conjunto_soluciones_pd.sort_values("of", ascending=False)

,instance,sol,of
55,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 96, 13, 15, 51, 20, 86, 55, 59, 93}",354.25
34,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 37, 90, 59, 12, 20, 26, 91, 63}",351.82
358,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18
181,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53
31,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 78, 15, 83, 53, 56, 61, 94}",349.45
...,...,...,...
409,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{34, 74, 75, 12, 82, 23, 21, 55, 88, 89}",317.91
191,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{32, 1, 3, 4, 70, 8, 79, 84, 54, 30}",317.78
562,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 39, 44, 77, 18, 55, 54, 87, 21, 30}",316.04
653,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{0, 71, 9, 43, 77, 78, 79, 21, 24, 31}",312.78


In [21]:
sol_max = conjunto_soluciones_pd[conjunto_soluciones_pd["of"] == max(conjunto_soluciones_pd["of"])].iloc[0,1]
conjunto_soluciones_pd["n_distintos"] = conjunto_soluciones_pd["sol"].map(lambda x: len(x.difference(sol_max)))
conjunto_soluciones_pd[conjunto_soluciones_pd["n_distintos"] > 0].sort_values("of", ascending=False)

,instance,sol,of,n_distintos
34,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 37, 90, 59, 12, 20, 26, 91, 63}",351.82,8
358,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18,5
181,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53,10
31,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 78, 15, 83, 53, 56, 61, 94}",349.45,9
145,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 15, 16, 20, 86, 59, 60, 93}",349.36,5
...,...,...,...,...
409,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{34, 74, 75, 12, 82, 23, 21, 55, 88, 89}",317.91,9
191,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{32, 1, 3, 4, 70, 8, 79, 84, 54, 30}",317.78,10
562,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{1, 39, 44, 77, 18, 55, 54, 87, 21, 30}",316.04,9
653,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{0, 71, 9, 43, 77, 78, 79, 21, 24, 31}",312.78,10


In [22]:
sol_0 = conjunto_soluciones[14]
sol_1 = conjunto_soluciones[57]

In [23]:
def transicion(sol_0, sol_1):
    best = None
    resultado = 0
    sol_2 = deepcopy(sol_0)
    sol_guia = deepcopy(sol_1)
    sol_guia["sol"] = sol_guia["sol"].difference(sol_0["sol"])
    for s0 in sol_0["sol"]:
        for s1 in sol_guia["sol"]:
            if s1 not in sol_0["sol"]:
                sol_2["sol"].remove(s0)
                sol_2["sol"].add(s1)
                sol_2["of"] = evaluate(sol_2)
                if best is None or sol_2["of"] > best["of"]:
                    anyadido = s1
                    best = deepcopy(sol_2)
                sol_2["sol"].remove(s1)
                sol_2["sol"].add(s0)
    
    sol_guia["sol"].remove(anyadido)

    if len(sol_guia["sol"]) > 1:
        resultado = transicion(best, sol_guia)

        if resultado["of"] < best["of"]:
            resultado = best
    else:
        print(best["sol"])
        resultado = best
    

    return resultado

best = transicion(sol_0, sol_1)

{97, 6, 27, 14, 79, 21, 23, 57, 59, 29}


In [24]:
best["of"]

331.25000000000006

In [25]:
sol_0["of"]

341.29999999999995

In [26]:
sol_1["sol"]

{9, 32, 33, 37, 47, 51, 55, 59, 64, 85}

### Opción para el Path Relinking

In [27]:
def relink(inicio, guia):
    current = deepcopy(inicio)
    best = deepcopy(inicio)

    to_add = guia["sol"] - current["sol"]
    to_remove = current["sol"] - guia["sol"]

    while to_add:
        vecinos = []
        for a in to_add:
            for r in to_remove:
                nuevo = deepcopy(current)
                nuevo["sol"].remove(r)
                nuevo["sol"].add(a)
                nuevo["of"] = evaluate(nuevo)
                vecinos.append(nuevo)
        if not vecinos:
            break
        current = max(vecinos, key=lambda s: s["of"])
        if current["of"] > best["of"]:
            best = deepcopy(current)

        to_add = guia["sol"] - current["sol"]
        to_remove = current["sol"] - guia["sol"]

    return best


In [28]:
# Seleccionamos las 5 mejores soluciones distintas (se puede modificar el nº de soluciones que se toman)
elite_df = conjunto_soluciones_pd.sort_values("of", ascending=False).drop_duplicates("sol").head(5)
elite_solutions = elite_df.to_dict("records")

# Aplicamos Path Relinking entre todos los pares de soluciones
mejor_pr = None
for i in range(len(elite_solutions)):
    for j in range(i+1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        for cand in [pr1, pr2]:
            if mejor_pr is None or cand["of"] > mejor_pr["of"]:
                mejor_pr = cand

# Mostramos la mejor solución obtenida por Path Relinking
print("Mejor solución encontrada por Path Relinking:")
print("Conjunto:", mejor_pr["sol"])
print("Valor objetivo:", mejor_pr["of"])


Mejor solución encontrada por Path Relinking:
Conjunto: {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}
Valor objetivo: 354.25


In [ ]:
# Repetimos el bucle anterior, aunque esta vez para guardar todas las soluciones
soluciones_pr = []

for i in range(len(elite_solutions)):
    for j in range(i + 1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        soluciones_pr.extend([pr1, pr2])

# Convertimos a DataFrame para análisis posterior
df_pr = pd.DataFrame(soluciones_pr).drop_duplicates(subset="sol")
print("\nTop 5 soluciones PR:")
print(df_pr.sort_values("of", ascending=False).head(5))

print(f"\nTotal de soluciones generadas por PR: {len(df_pr)}")

                                             instance  \
0   {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
8   {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
14  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   
18  {'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...   

                                         sol      of  n_distintos  
0   {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}  354.25            0  
8    {98, 2, 26, 37, 91, 12, 20, 90, 59, 63}  351.82            8  
14  {64, 70, 43, 79, 16, 20, 86, 59, 60, 93}  350.18            5  
18    {5, 70, 39, 8, 74, 43, 14, 79, 17, 60}  349.53           10  

Total de soluciones generadas por PR: 4
